# Overview

This notebook demostrates the usage of `piper_control`, our lightweight wrapper
of `piper_sdk` for controlling AgileX Piper robots.

The `piper_sdk` API is powerful and quickly maturing, but it's a bit complex and
under-documented, and we found it helpful to define a simple abstraction for
basic I/O.

There are also several sharp bits in `piper_sdk` which can make the robots seem
tempermental, e.g. becoming unresponsive despite repeated calls to
`MotionCtrl_2`, `EnableArm`, `GripperCtrl`, etc. We've bundled our solutions
into `PiperControl` so `reset` and the various move commands perform as one
would expect.

In [ ]:
# Enable autoreload (optioal, use if modifying piper_control.py)
%load_ext autoreload
%autoreload 2

# CAN Connect
This section demonstrates the use of `piper_connect` to establish the CAN
connection to the robot(s).

It is a python counterpart to AgileX's `can_activate.sh`.

## Grant sudo access
This cell is only required for granding sudo to `piper_connect.activate`.
For deployed robot machines consider adding your user to sudoers file:

```bash
sudo visudo
# Add this line at the end:
<your username> ALL=(ALL) NOPASSWD:ALL
```

In [ ]:
import subprocess
import getpass

def sudo_warmup():
    try:
        print("🔐 Sudo setup for Jupyter...")
        pw = getpass.getpass("Enter your password for sudo: ")
        proc = subprocess.run(
            ["sudo", "-S", "echo", "sudo access granted"],
            input=pw + "\n", text=True, check=True, capture_output=True
        )
        print(proc.stdout.strip())
    except subprocess.CalledProcessError as e:
        print("❌ Sudo failed:", e.stderr.strip())

sudo_warmup()


## Poll and Connect

In [ ]:
from piper_control import piper_connect

In [ ]:
ports = piper_connect.find_ports()
print(ports)

In [ ]:
piper_connect.activate()
print(piper_connect.active_ports())

# Initialize and Query State

In [ ]:

from piper_control import piper_control

robot = piper_control.PiperControl(can_port="can0")

# Resets the robot and enables the motors and motion controller for the arm.
# This call is necessary to be able to both query state and send commands to the
# robot.
robot.reset()

print(robot.get_joint_positions())
print(robot.get_joint_velocities())
print(robot.get_joint_efforts())
print(robot.get_gripper_state())

In [ ]:
# Prints out a higher level status of the robot.
print(robot.get_status())

# Enabling and Resetting the Robot

In [ ]:
# Full reset, clearing all error codes.
robot.reset()

In [ ]:
# Can also reset it without re-enabling the arm or motion control.
# Note that it won't move until it is re-enabled (see below).
robot.reset(enable_arm=False, enable_motion=False)
# TODO(akhil): Add output to this when both are false

In [ ]:
robot.enable()

In [ ]:
robot.disable()

## Move a joint

In [ ]:
joint_angles = robot.get_joint_positions()
joint_angles[-2] -= 0.1
print(f"setting joint angles to {joint_angles}")
robot.set_joint_positions(joint_angles)

# Send Gripper Commands

In [ ]:
robot.set_gripper_ctrl(piper_control.GRIPPER_ANGLE_MAX * 0.9, piper_control.GRIPPER_EFFORT_MAX * 0.5)

In [ ]:
robot.set_gripper_ctrl(piper_control.GRIPPER_ANGLE_MAX * 0.1, piper_control.GRIPPER_EFFORT_MAX * 0.5)

## Move to special positions

In [ ]:
robot.set_joint_positions(piper_control.REST_POSITION)
# TODO(jscholz): Revisit the rest position.

In [ ]:
robot.set_joint_positions(piper_control.DOWN_POSITION)

## Cartesian Control

In [ ]:
ee_pose = robot.get_end_effector_pose()
ee_pose[0] += 0.02  # Move X by 5 cm
print(f"Sending updated pose: {ee_pose}")
robot.set_cartesian_position(ee_pose)

## Puppeteering

In [ ]:
robot0 = piper_control.PiperControl(can_port="can0")
robot1 = piper_control.PiperControl(can_port="can1")

In [ ]:
robot0.reset()
robot1.reset()

In [ ]:
robot0.set_joint_positions(piper_control.REST_POSITION)

In [ ]:
robot0.set_joint_positions(piper_control.DOWN_POSITION)

In [ ]:
robot1.set_joint_positions(piper_control.REST_POSITION)

In [ ]:
robot1.set_joint_positions(piper_control.DOWN_POSITION)

In [ ]:
import time
while True:
  joint_angles = robot1.get_joint_positions()
  print(f"{joint_angles=}")
  time.sleep(0.01)
  robot0.set_joint_positions(joint_angles)